In [1]:
import torch
import torch.nn.functional as F
import modules_rwkv

In [2]:
w = torch.rand((1024, 124)).to("cuda")
k = torch.rand((1024, 124)).to("cuda")
v = torch.rand((1024, 124)).to("cuda")
r = torch.rand((1024, 124)).to("cuda")
u = torch.rand(124).to("cuda")